In [1]:
from utils import *
import os.path
from scipy.stats import qmc
from tqdm import tqdm
from pymoo.operators.sampling.lhs import LHS
import polars as pl

In [2]:
save_dir = 'scale_2_problem'
create_directory_if_not_exist(save_dir)

In [3]:
sample_size = 250 #times dimention
problem_dim = 5

In [4]:
suite = list(generate_problems(problem_dim))

In [5]:
file = f'{save_dir}/samples__dim_{problem_dim}__samplesize_{sample_size}.parquet'

In [6]:
if os.path.isfile(file)==False:
    data = {'problem': [], 'instance': [], 'y':[], 'optimum':[]}
    
    for problem in tqdm(suite):
        sampling = LHS()
        sampler = qmc.LatinHypercube(d=problem.n_var)
        sample = sampler.random(n=sample_size*problem.n_var)
        sample_scaled = qmc.scale(sample, problem.xl, problem.xu)
        data['problem'].extend(len(sample_scaled)*[problem.id_function])
        data['instance'].extend(len(sample_scaled)*[problem.id_instance])

        for r in range(problem.n_var):
            if f'x_{r}' not in data:
                data[f'x_{r}'] = []
            data[f'x_{r}'].extend(sample_scaled[:, r])

        data['y'].extend(problem.evaluate(sample_scaled).flatten())
        
        opt = problem.evaluate(problem.pareto_set())[0, 0]
        data['optimum'].extend(len(sample_scaled)*[opt])
        
        
        
        
    df = pl.DataFrame(data)
    df.write_parquet(file)
else:
    df = pl.read_parquet(file)

100%|██████████| 360/360 [00:02<00:00, 133.44it/s]


In [7]:
df

problem,instance,y,optimum,x_0,x_1,x_2,x_3,x_4
i64,i64,f64,f64,f64,f64,f64,f64,f64
1,1,124.841716,79.48,-1.228183,4.571686,0.715115,4.304565,-4.301335
1,1,95.411072,79.48,-1.958272,1.043912,-2.906042,2.421185,-1.588564
1,1,142.313325,79.48,3.926394,2.681122,-0.636988,2.380795,3.183869
1,1,165.046834,79.48,-4.229127,2.787666,-4.49542,-0.453723,2.799324
1,1,145.549628,79.48,-3.550885,-4.083994,-2.119983,2.793942,3.669928
…,…,…,…,…,…,…,…,…
24,15,246.884869,149.81,1.347734,-2.153294,-1.176135,2.364704,-0.516795
24,15,372.082841,149.81,4.87877,1.051937,-1.893373,-4.335428,-4.617713
24,15,394.905955,149.81,-3.724222,-3.088483,4.794444,-3.299469,3.018012
